### SAT Visit Sweden National API

* [Issue #222](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/222)
* This Notebook [222_SAT_Visit_Sweden_national_API.ipynb]()

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-10-05 09:45:54


In [4]:
import folium
import geopandas as gpd
import requests
import os, datetime, re
from shapely.geometry import Point

# ---------------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------------
SAT_PATH = "SAT_full.geojson"
OUTPUT_PREFIX = "222_SAT_Visit_Ssweden"
VISIT_SWEDEN_API = "https://data.visitsweden.com/store/search"

# ---------------------------------------------------------------------
# Read SAT trail and create 1000 m buffer
# ---------------------------------------------------------------------
sat = gpd.read_file(SAT_PATH).to_crs("EPSG:3006")
sat_buffer = sat.buffer(1000).union_all()
sat_buffer_gdf = gpd.GeoDataFrame(geometry=[sat_buffer], crs="EPSG:3006").to_crs("EPSG:4326")
sat = sat.to_crs("EPSG:4326")

# ---------------------------------------------------------------------
# Fetch Visit Sweden tourism data
# ---------------------------------------------------------------------
def fetch_visit_sweden(limit=400):
    """Fetch public tourism items (lodging, attractions, restaurants)"""
    query = (
        "public:true AND ("
        "rdfType:http\\:\\/\\/schema.org\\/TouristAttraction "
        "OR rdfType:http\\:\\/\\/schema.org\\/LodgingBusiness "
        "OR rdfType:http\\:\\/\\/schema.org\\/Restaurant)"
    )
    params = {
        "type": "solr",
        "query": query,
        "limit": limit,
        "offset": 0,
        "rdfFormat": "application/ld+json",
    }
    r = requests.get(VISIT_SWEDEN_API, params=params)
    r.raise_for_status()
    return r.json()

print("Fetching data from Visit Sweden API ...")
data = fetch_visit_sweden(limit=400)
print(f"Fetched {len(data)} entries")

# ---------------------------------------------------------------------
# Parse data safely
# ---------------------------------------------------------------------
def parse_geo(obj):
    """Return (lat, lon) if geo info exists and is valid"""
    if not obj:
        return None
    if isinstance(obj, dict):
        lat = obj.get("latitude")
        lon = obj.get("longitude")
        if lat and lon:
            return float(lat), float(lon)
    elif isinstance(obj, list) and obj and isinstance(obj[0], dict):
        lat = obj[0].get("latitude")
        lon = obj[0].get("longitude")
        if lat and lon:
            return float(lat), float(lon)
    elif isinstance(obj, str):
        m = re.match(r"([0-9.+-]+)[ ,;]([0-9.+-]+)", obj)
        if m:
            return float(m.group(1)), float(m.group(2))
    return None

features = []
for d in data:
    geo = parse_geo(d.get("geo"))
    if not geo:
        continue
    lat, lon = geo
    name = d.get("name", ["(no name)"])
    if isinstance(name, list):
        name = name[0]
    desc = d.get("description", [""])
    if isinstance(desc, list):
        desc = desc[0]
    url = d.get("url", [""])
    if isinstance(url, list):
        url = url[0]
    typ = d.get("@type", "")
    features.append({
        "name": name,
        "type": typ,
        "description": desc,
        "url": url,
        "geometry": Point(lon, lat)
    })

vs_gdf = gpd.GeoDataFrame(features, crs="EPSG:4326")
print(f"Valid geocoded entries: {len(vs_gdf)}")

# Spatial filter: only keep entries inside 1000m buffer
within_buffer = gpd.sjoin(vs_gdf, sat_buffer_gdf, predicate="intersects")
print(f"Entries within 1 km buffer: {len(within_buffer)}")

# ---------------------------------------------------------------------
# Color/group by schema.org type
# ---------------------------------------------------------------------
def icon_color(schema_type):
    if "LodgingBusiness" in schema_type:
        return "red"
    if "Restaurant" in schema_type:
        return "green"
    if "TouristAttraction" in schema_type:
        return "blue"
    return "gray"

# ---------------------------------------------------------------------
# Build map
# ---------------------------------------------------------------------
m = folium.Map(location=[59.4, 18.5], zoom_start=9, tiles="OpenStreetMap")

# SAT Trail
fg_trail = folium.FeatureGroup(name="SAT Trail")
folium.GeoJson(
    sat, style_function=lambda x: {"color": "blue", "weight": 3},
    tooltip="SAT Trail"
).add_to(fg_trail)
fg_trail.add_to(m)

# Buffer (1 km)
fg_buffer = folium.FeatureGroup(name="SAT Buffer (1000 m)")
folium.GeoJson(
    sat_buffer_gdf,
    style_function=lambda x: {"color": "green", "fillColor": "green", "fillOpacity": 0.2, "weight": 1},
    tooltip="1 km Buffer"
).add_to(fg_buffer)
fg_buffer.add_to(m)

# Visit Sweden POIs
fg_vs = folium.FeatureGroup(name="Visit Sweden API")
for _, row in within_buffer.iterrows():
    name = row["name"]
    desc = row["description"] or ""
    url = row["url"]
    typ = row["type"]
    html = f"<b>{name}</b><br><i>{typ}</i><br>{desc}<br>"
    if url:
        html += f'<a href="{url}" target="_blank">🌐 Visit page</a>'
    folium.Marker(
        [row.geometry.y, row.geometry.x],
        tooltip=f"{name} ({typ})",
        popup=folium.Popup(html, max_width=400),
        icon=folium.Icon(color=icon_color(typ), icon="info-sign")
    ).add_to(fg_vs)
fg_vs.add_to(m)

# Legend
legend_html = """
<div style="position: fixed; bottom: 40px; left: 10px; width: 160px; 
     background: white; border:2px solid grey; z-index:9999; font-size:12px;
     border-radius:8px; padding:6px;">
<b>Legend</b><br>
<span style="color:red;">■</span> LodgingBusiness<br>
<span style="color:green;">■</span> Restaurant<br>
<span style="color:blue;">■</span> TouristAttraction<br>
<span style="color:gray;">■</span> Other
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Layer control
folium.LayerControl(collapsed=False).add_to(m)

# About box
add_about_box(m, issue_number=222, map_name="SAT Visit Sweden API")

# ---------------------------------------------------------------------
# Save outputs
# ---------------------------------------------------------------------
os.makedirs("output", exist_ok=True)
ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")
m.save(out_ts)
m.save(out_latest)

print("Saved:")
print(" ", out_ts)
print(" ", out_latest)

m


Fetching data from Visit Sweden API ...
Fetched 5 entries


AttributeError: 'str' object has no attribute 'get'

In [2]:
from datetime import datetime
# End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen
now = datetime.now()
# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-10-05 09:45:54
Total time elapsed: 00 minutes 00.00 seconds
